In [23]:
# load the data
import tarfile
import os.path
import json
import re
from bz2 import BZ2File
from urllib import request
from io import BytesIO

import numpy as np


fname = "cmv.tar.bz2"
# url = "https://chenhaot.com/data/cmv/" + fname

# download if not exists
# if not os.path.isfile(fname):
#     f = BytesIO()
#     with request.urlopen(url) as resp, open(fname, 'wb') as f_disk:
#         data = resp.read()
#         f_disk.write(data)  # save to disk too
#         f.write(data)
#         f.seek(0)
# else:
f = open(fname, 'rb')


tar = tarfile.open(fileobj=f, mode="r")

# Extract the file we are interested in

train_fname = "op_task/train_op_data.jsonlist.bz2"
test_fname = "op_task/heldout_op_data.jsonlist.bz2"

all_train_fname = "all/train_period_data.jsonlist.bz2"
all_test_fname = "all/heldout_period_data.jsonlist.bz2"

pair_train_fname = "pair_task/train_pair_data.jsonlist.bz2"
pair_test_fname = "pair_task/heldout_pair_data.jsonlist.bz2"

train_bzlist = tar.extractfile(train_fname)
all_train_bzlist = tar.extractfile(all_train_fname)
pair_train_bzlist = tar.extractfile(pair_train_fname)

# Deserialize the JSON list
original_posts_train = [
    json.loads(line.decode('utf-8'))
    for line in BZ2File(train_bzlist)
]
all_train = [
    json.loads(line.decode('utf-8'))
    for line in BZ2File(all_train_bzlist)
]
pair_train = [
    json.loads(line.decode('utf-8'))
    for line in BZ2File(pair_train_bzlist)
]
test_bzlist = tar.extractfile(test_fname)
all_test_bzlist = tar.extractfile(all_test_fname)
pair_test_bzlist = tar.extractfile(pair_test_fname)

original_posts_test = [
    json.loads(line.decode('utf-8'))
    for line in BZ2File(test_bzlist)
]
all_test = [
    json.loads(line.decode('utf-8'))
    for line in BZ2File(all_test_bzlist)
]
pair_test = [
    json.loads(line.decode('utf-8'))
    for line in BZ2File(pair_test_bzlist)
]
f.close()

In [24]:
print(f'{len(original_posts_train)} + {len(original_posts_test)} = {len(original_posts_train + original_posts_test)}')
print(f'{len(all_train)} + {len(all_test)} = {len(all_train + all_test)}')
print(f'{len(pair_train)} + {len(pair_test)} = {len(pair_train + pair_test)}')

10743 + 1529 = 12272
18363 + 2263 = 20626
3456 + 807 = 4263


In [26]:
complete_key_set = []
for a in all_train:
    complete_key_set.extend(list(a.keys()))
complete_key_set = set(complete_key_set)
print(complete_key_set)

{'is_self', 'created_utc', 'ups', 'edited', 'secure_media_embed', 'media', 'link_flair_text', 'media_embed', 'report_reasons', 'visited', 'banned_by', 'stickied', 'comments', 'downs', 'selftext', 'secure_media', 'suggested_sort', 'over_18', 'thumbnail', 'mod_reports', 'approved_by', 'selftext_html', 'hidden', 'name', 'created', 'num_comments', 'author_flair_css_class', 'distinguished', 'clicked', 'id', 'domain', 'link_flair_css_class', 'subreddit_id', 'num_reports', 'saved', 'title', 'user_reports', 'archived', 'author_flair_text', 'author', 'subreddit', 'permalink', 'score', 'likes', 'url', 'gilded'}


In [27]:
def attch_to_parent(current, tree_dict, layer):
    # BFS search
    if current['parent_id'] in tree_dict:
        tree_dict[current['parent_id']][current['name']] = {}
        return None
    else:
        for k, v in tree_dict.items():
#             print(f"searching for {current['parent_id']} in {v.keys()}")
            attch_to_parent(current, v, layer + 1)

def print_tree(tree_dict, layer, orig, root):
    if tree_dict == {}:
        print('======================== end of subtree ========================')
        return
    else:
        for k, v in tree_dict.items():
            print(f'{" ".join(["*"] * layer)} {k}')
            if k != root:
                print(orig[k])
            print_tree(v, layer + 3, orig, root)

In [28]:
def show_comments(cmv_post):
    root = cmv_post['name']
    if 'comments' not in cmv_post:
        print('No comments found')
    else:
        print(len(cmv_post['comments']))
        tree_dict = {root: {}}
        for i in range(len(cmv_post['comments'])):
            current = cmv_post['comments'][i]
            attch_to_parent(current, tree_dict, 0)
        comments_dict = {c['name']: c['author'] + '\t' + c['body'] for c in cmv_post['comments']}
        print_tree(tree_dict, 0, comments_dict, root)


In [29]:
show_comments(all_train[0])

7
 t3_2rpvc7
* * * t1_cnibo8i
Kman17	I'd be a little bit cautious of removing the self-policing elements out of the sport.  A bench clearing brawl is usually retaliation for a headhunting pitcher.  

If you come down hard on retaliation, you then have to come down hard on the instigation as well... and before you know it, the sport itself is radically different.

Just look at the NBA.  The NBA, mostly after Jordan in the late 90's / early 00's, came down hard on player altercations.  This was for a variety of reasons mostly related to image / sportsmanship (and the racial double-standards put on mostly black players, but I digress) and to a lesser extent injuries.  

The occasional scuffles of the 80's / 90's became unheard of... and following that, the league came down hard on "chippy" fouls, and now they hand out double technicals any time anyone looks at anyone funny.  The end result is that reckless and sloppy charging to the basket, and relying on the superstar treatment to get to

In [30]:
# sanity check
for a in all_train[0]['comments']:
    print(f"{a['name']}\t{a['parent_id']}\t{a['replies']}")

t1_cnibo8i	t3_2rpvc7	{'kind': 'Listing', 'data': {'modhash': '', 'children': ['cnikfa7'], 'after': None, 'before': None}}
t1_cnikfa7	t1_cnibo8i	
t1_cni5tm6	t3_2rpvc7	
t1_cni6vol	t3_2rpvc7	{'kind': 'Listing', 'data': {'modhash': '', 'children': ['cni7f98'], 'after': None, 'before': None}}
t1_cni7f98	t1_cni6vol	{'kind': 'Listing', 'data': {'modhash': '', 'children': ['cniszwq'], 'after': None, 'before': None}}
t1_cniszwq	t1_cni7f98	{'kind': 'Listing', 'data': {'modhash': '', 'children': ['cnjaebu'], 'after': None, 'before': None}}
t1_cnjaebu	t1_cniszwq	


In [46]:
malleable_op_train_count = 0

for dt in all_train:
    for each in dt['comments']:
        if 'author' in each:
            if each['author'] == 'DeltaBot' and each['body'].startswith('Confirmed: 1 delta awarded to'):
                malleable_op_train_count += 1
                break

print(f'malleable records: {malleable_op_train_count}, total records: {len(all_train)}, percentage: {malleable_op_train_count / len(all_train)}')


malleable records: 4837, total records: 18363, percentage: 0.2634101181724119


In [48]:
malleable_op_test_count = 0

for dt in all_test:
    for each in dt['comments']:
        if 'author' in each:
            if each['author'] == 'DeltaBot' and each['body'].startswith('Confirmed: 1 delta awarded to'):
                malleable_op_test_count += 1
                break

print(f'malleable records: {malleable_op_test_count}, total records: {len(all_test)}, percentage: {malleable_op_test_count / len(all_test)}')


malleable records: 1047, total records: 2263, percentage: 0.4626601855943438


In [49]:
malleable_op = [post["name"] for post in original_posts_train if post["delta_label"]]
print(f'{len(original_posts_train)}, {len(malleable_op)}')

10743, 3191


In [50]:
for at in all_train:
    if at['name'] in malleable_op:
        show_comments(at)
#         print(at)
        break

30
 t3_2ro0ti
* * * t1_cnhpddf
Legomystrudel	May or may not have any sort of healing properties, but they smell really good.
* * * * * * t1_cnhpqan
GoldenTaint	Most do.  Some smell kinda funky.
======================== end of subtree ========================
* * * t1_cnhuxye
Bradm77	[This answer in /r/askscience does a pretty good job of describing essential oils.](http://www.reddit.com/r/askscience/comments/2pp57r/how_effective_are_essential_oils/cmytzdq) As the comment points out there are benefits to essential oils if they are used in the right way.  However, the current fad mixes the raw essential oils with other stuff that may or may not be good for you.  The raw essential oils are actually "a vital resource for drug discovery."
* * * * * * t1_cni1m79
GoldenTaint	&amp;#8710; While my view may not have shifted enough for me to avoid pissing my wife off, I have to say that you really brought home the bacon on this one and gave me some hope for the oils.  thank you for your response.

In [51]:
from IPython.display import Markdown


def show_post(cmv_post):
    md_format = "**{title}** \n\n {selftext}".format(**cmv_post)
    md_format = "\n".join(["> " + line for line in md_format.splitlines()])
    return Markdown(md_format)

In [52]:
show_post(original_posts_train[200])

> **CMV: Taxation should be voluntary, with individuals retaining the right to opt out of it.** 
> 
>  Let me preface this argument by saying that I don't particularly agree with the Tea Party. I'm a Democrat who votes for candidates that support ending the poverty trap, expanding social services, and wish to improve fundamental institutions like education or military service. With that said, I also believe in the concept of personal liberty. This includes the right to refuse taxation upon the basis of... well, whatever the fuck you want.
> 
> I know what your thinking, "But how could we fund things like food stamps if shitty people are like 'fuck those lazy blacks that suck up muh moneyz'?" However, in order to participate in the American system of democracy, one must agree to pay taxes unconditionally. A citizen ought not be able to decide which services he or she pays for. Instead, one can only receive protection from, say, the police if they pay every tax mandated by the local, state, or federal government.
> In my opinion, this would create a voluntary political system where individual rights are maintained. If we believe that the government overwhelmingly improves our own conditions, as I do, then only a small amount of citizens would opt out of public services because they disagree with certain elements of policy. Within the framework of government existing because it benefits the people, no one should expect a sudden drop in tax revenues because citizens are intelligent enough to understand how their own interests are promoted by the state on balance.
> 
> So go forth and CMV, noble redditors!
> _____
> 
> &gt; *Hello, users of CMV! This is a footnote from your moderators. We'd just like to remind you of a couple of things. Firstly, please remember to* ***[read through our rules](http://www.reddit.com/r/changemyview/wiki/rules)***. *If you see a comment that has broken one, it is more effective to report it than downvote it. Speaking of which,* ***[downvotes don't change views](http://www.reddit.com/r/changemyview/wiki/guidelines#wiki_upvoting.2Fdownvoting)****! If you are thinking about submitting a CMV yourself, please have a look through our* ***[popular topics wiki](http://www.reddit.com/r/changemyview/wiki/populartopics)*** *first. Any questions or concerns? Feel free to* ***[message us](http://www.reddit.com/message/compose?to=/r/changemyview)***. *Happy CMVing!*

In [53]:
def cleanup(cmv_post):
    lines = [line for line in cmv_post.splitlines()
             if not line.lstrip().startswith("&gt;")
             and not line.lstrip().startswith("____")
             and "edit" not in " ".join(line.lower().split()[:2])
            ]
    return "\n".join(lines)

In [54]:
show_post(dict(title=original_posts_train[200]['title'],
               selftext=cleanup(original_posts_train[200]['selftext'])))

> **CMV: Taxation should be voluntary, with individuals retaining the right to opt out of it.** 
> 
>  Let me preface this argument by saying that I don't particularly agree with the Tea Party. I'm a Democrat who votes for candidates that support ending the poverty trap, expanding social services, and wish to improve fundamental institutions like education or military service. With that said, I also believe in the concept of personal liberty. This includes the right to refuse taxation upon the basis of... well, whatever the fuck you want.
> 
> I know what your thinking, "But how could we fund things like food stamps if shitty people are like 'fuck those lazy blacks that suck up muh moneyz'?" However, in order to participate in the American system of democracy, one must agree to pay taxes unconditionally. A citizen ought not be able to decide which services he or she pays for. Instead, one can only receive protection from, say, the police if they pay every tax mandated by the local, state, or federal government.
> In my opinion, this would create a voluntary political system where individual rights are maintained. If we believe that the government overwhelmingly improves our own conditions, as I do, then only a small amount of citizens would opt out of public services because they disagree with certain elements of policy. Within the framework of government existing because it benefits the people, no one should expect a sudden drop in tax revenues because citizens are intelligent enough to understand how their own interests are promoted by the state on balance.
> 
> So go forth and CMV, noble redditors!

In [55]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score, accuracy_score

In [56]:
pair_train[0]['positive']['comments'][0]['body']

'Look at the definition you provided, if we remove the exclusion of things which humans create:\n\n&gt; existing in nature ~~and not made or caused by people~~\n\nSo essentially, by this definition, "natural things" are "things that exist," which is frankly rather meaningless. If one wanted to discuss the results of human activity we would then have to make up a new word which could be redefined by the same argument. \n\nThe whole point of the word is to exclude human activity. If you remove that aspect, it simply ceases to have meaning.'

In [57]:
# prepare the data in Python list form

text_train = [
#     cleanup(tr['positive']['comments'][0]['body'])
    cleanup(tr['selftext'])
    for tr
    in original_posts_train
]

text_test = [
#     cleanup(te['positive']['comments'][0]['body'])
    cleanup(te['selftext'])
    for te
    in original_posts_test
]

In [58]:
text_train[:3]

["I think the world is automating fast enough that a utopia will arise where no one will have to work anymore. Within the next 2 decades or so, having a job won't mean much, and most people will be artists and scientists. \n\nMy parents let me live with them, so I can just wait until the utopia happens.\n\nCMV.",
 "First off, I do not believe that Iran *should* have nuclear weapons. In fact, I believe Iran having nuclear weapons makes the world less safe overall. However, I believe that as a sovereign nation they have the right to develop nuclear weapons if they so choose.\n\nWhy do I believe this:\n\n1. It is in Iran's best strategic interests to develop nuclear weapons in order to counter Israel (which has nuclear weapons), and additionally to one-up Saudi Arabia (their main regional rival), and guarantee their safety against other Arab nations with whom they have historically had rocky relations (Iraq, for example).\n2. If Israel can illegally possess nuclear weapons (they haven't o

In [59]:
vect = TfidfVectorizer(use_idf=False, norm='l1')
X_train = vect.fit_transform(text_train)

In [61]:
# print(X_train[0])

In [62]:
is_malleable = np.array([post["delta_label"] for post in original_posts_train])

In [22]:

lr_models = []
params = [('l1', 'liblinear'), ('l2', 'sag')]
for penalty, solver in params:
    print(f'LR with {penalty} penalty')
    lr = LogisticRegressionCV(Cs=10, class_weight='balanced',
                          scoring='roc_auc', solver=solver,
                          tol=0.001, max_iter=1000,
                          random_state=0, cv=5, penalty=penalty)
    lr.fit(X_train, is_malleable)

    X_test = vect.transform(text_test)
    is_malleable_test = np.array([post["delta_label"] for post in original_posts_test])
    test_pred = lr.predict_proba(X_test)[:, 1]
    test_roc = roc_auc_score(is_malleable_test, test_pred)
    print("Test ROC AUC score: {:.3f}".format(test_roc))
    lr_models.append(lr)

LR with l1 penalty
Test ROC AUC score: 0.543
LR with l2 penalty
Test ROC AUC score: 0.531


In [22]:
def top_features(coef, words, n=15):
    order = np.argsort(-coef)
    for i_hi, i_lo in zip(order[:n], order[:-n - 1:-1]):
        row_vals = coef[i_hi], words[i_hi], coef[i_lo], words[i_lo]
        print("[{:>4.1f}] {:<20}[{:>4.1f}] {:<20}".format(*row_vals))

In [23]:
top_features(lr.coef_.flatten(), vect.get_feature_names_out())

[ 8.9] so                  [-9.1] its                 
[ 7.2] even                [-5.5] having              
[ 5.6] feel                [-5.3] anyone              
[ 5.5] would               [-5.2] government          
[ 5.5] being               [-5.2] right               
[ 5.4] more                [-5.2] anything            
[ 5.4] please              [-5.2] our                 
[ 5.3] hard                [-5.0] certain             
[ 5.3] doesn               [-5.0] nothing             
[ 5.3] ll                  [-5.0] world               
[ 5.1] when                [-5.0] minimum             
[ 5.0] system              [-4.7] we                  
[ 4.7] philosophy          [-4.7] men                 
[ 4.5] subject             [-4.6] life                
[ 4.4] help                [-4.6] population          


### Number of words

In [63]:
text_train[0]

"I think the world is automating fast enough that a utopia will arise where no one will have to work anymore. Within the next 2 decades or so, having a job won't mean much, and most people will be artists and scientists. \n\nMy parents let me live with them, so I can just wait until the utopia happens.\n\nCMV."

In [64]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

In [65]:
import re

pron_i = ['ive',
 "i'd",
 "i'm",
 'myself',
 'mine',
 'me',
 'im',
 "i'll",
 'i',
 "i've",
 'id',
 'my']

pron_we = ["we'd",
 "we're",
 'lets',
 'weve',
 'we',
 'our',
 'ourselves',
 "let's",
 'ours',
 'us',
 "we'll",
 "we've"]

pron_we_re = re.compile("\\b({})\\b".format("|".join(pron_we)), re.IGNORECASE)
pron_i_re = re.compile("\\b({})\\b".format("|".join(pron_i)), re.IGNORECASE)

In [66]:
def gen_x(text, posts, tokenizer):
    X_ = []
    for i, post in enumerate(posts):
        content = post["selftext"].lower()
        content = cleanup(content)
        denom = len(content.split())
        if denom:
            X_.append([
                len(tokenizer.tokenize(text[i])),
                len(pron_we_re.findall(content)) / denom, 
                len(pron_i_re.findall(content)) / denom
            ])
        else:
            X_.append([
                len(tokenizer.tokenize(text[i])),
                0, 
                0
            ])
    return X_

In [67]:
X_train = gen_x(text_train, original_posts_train, tokenizer)
X_test = gen_x(text_test, original_posts_test, tokenizer)

In [68]:
lr_models = []
params = [('l1', 'liblinear'), ('l2', 'sag')]
for penalty, solver in params:
    print(f'LR with {penalty} penalty')
    lr = LogisticRegressionCV(Cs=10, class_weight='balanced',
                          scoring='roc_auc', solver=solver,
                          tol=0.001, max_iter=1000,
                          random_state=0, cv=5, penalty=penalty)
    lr.fit(X_train, is_malleable)

    is_malleable_test = np.array([post["delta_label"] for post in original_posts_test])
    test_pred = lr.predict_proba(X_test)[:, 1]
    test_roc = roc_auc_score(is_malleable_test, test_pred)
    print("Test ROC AUC score: {:.3f}".format(test_roc))

    lr_models.append(lr)

LR with l1 penalty
Test ROC AUC score: 0.524
LR with l2 penalty
Test ROC AUC score: 0.506
